In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("sagorsarker/bangla-bert-base")
model = AutoModelForMaskedLM.from_pretrained("sagorsarker/bangla-bert-base")



# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="sagorsarker/bangla-bert-base")

model.safetensors:  97%|#########6| 640M/660M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/sagorsarker/bangla-bert-base/187685debb02ef68d1f006babe89269074821c576de8b6f792f8811dd4c5ff6d?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1732469488&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjQ2OTQ4OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9zYWdvcnNhcmtlci9iYW5nbGEtYmVydC1iYXNlLzE4NzY4NWRlYmIwMmVmNjhkMWYwMDZiYWJlODkyNjkwNzQ4MjFjNTc2ZGU4YjZmNzkyZjg4MTFkZDRjNWZmNmQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=cMpYfpTxcUgODcp11Z94MxLOlhNw8ZtnRHK6NVej-eTixpEt7j%7EKBuiIMfK1brgfijvtKvCu370VhoqPoutdr8lRGcc9NIbt1YnTkYcR9wjomHNv3TmapINdRTJAw7rvfk89fbc56W9CqVmEQ5V01EckU76Ym3UusNw4rEQDhGA8JT0zZ35LOOA7FznHJQLfGNPPuBduFYG6lZYcMpLbj9GSwsHkYvUyScB6-USSuoaXDL%7Ef1hYFNuHrmaQsHRwl6GQUE%7EiOgnabhneXUhLRmQ777FDF-UVLJVHpmqrGtG0IdIy%7EsP9iH7xM1L0fk8fjAK1ukC3P7udGDYdif-tqQw__&Key-Pair-Id=K3RPWS32NSSJCE

model.safetensors:  97%|#########6| 640M/660M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at sagorsarker/bangla-bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.

In [ ]:


# Load model directly
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer_2 = AutoTokenizer.from_pretrained("csebuetnlp/banglabert")
model_2 = AutoModelForPreTraining.from_pretrained("csebuetnlp/banglabert")

In [ ]:
import torch
import random

# Move model to GPU if available
device = torch.device("mps")
model.to(device)

# Define a function for text generation with sampling
def generate_text(seed_text, max_length=20, top_k=10):
    generated_text = seed_text
    for _ in range(max_length):
        # Add a mask token at the end of the seed text
        input_text = generated_text + f" {tokenizer.mask_token}"
        
        # Tokenize input and ensure it's on the correct device
        inputs = tokenizer(input_text, return_tensors="pt").to(device)
        
        # Predict the masked token
        outputs = model(**inputs)
        logits = outputs.logits

        # Find the index of the [MASK] token
        mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

        # Get the logits for the masked token
        mask_token_logits = logits[0, mask_token_index, :].squeeze()

        # Apply top-k sampling
        top_k_logits, top_k_indices = torch.topk(mask_token_logits, top_k)
        probabilities = torch.softmax(top_k_logits, dim=-1)
        
        # Sample a token from the top-k logits
        sampled_token_id = random.choices(top_k_indices.tolist(), weights=probabilities.tolist(), k=1)[0]

        # Decode the predicted token and add it to the generated text
        predicted_token = tokenizer.decode([sampled_token_id]).strip()
        generated_text += " " + predicted_token

        # Stop if the prediction is a sentence-ending token
        if predicted_token in ["।", ".", "!"]:
            break
    
    return generated_text

# Example usage
seed = "আমি বাংলায় গান"
generated = generate_text(seed, max_length=20, top_k=10)
print("Generated Text:", generated)

Generated Text: আমি বাংলায় গান ।
